# LIBRARY

In [1]:
import semopy as sem
import pandas as pd
import numpy as np
import pingouin
import scipy as stats
import graphviz

# IMPORT DATA

In [2]:
mydatadf = pd.read_csv("edu.csv")
mydatadf.head(6)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,y1,y2,y3
0,5,5,6,6,5,4,6,7,5,5,6,7,7,7,7
1,6,6,5,6,6,6,6,6,6,4,7,6,7,7,7
2,6,6,6,6,6,6,6,6,6,7,7,7,7,6,6
3,1,2,1,1,3,2,1,1,1,1,1,2,1,1,1
4,7,6,7,7,7,7,6,7,7,7,7,7,7,7,7
5,7,7,7,6,6,7,6,7,6,6,6,6,7,6,7


# UJI NORMALITAS (UNIVARIATE DAN MULTIVARIATE)

In [3]:
# UNIVARIATE
# mengubah dataframe ke array
mydataarray = mydatadf.to_numpy()

In [4]:
# mendefinisikan dan menghitung Mahalanobis distance
def mahalanobis(x=None, data=None, cov=None):
    x_mu = x - np.mean(mydatadf)
    if not cov:
        cov = np.cov(mydatadf.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    return mahal.diagonal()

In [5]:
# menambahkan kolom 'mahalanobis' pada dataframe
mydatadf ['mahalanobis'] = mahalanobis(x=mydatadf, 
data=mydatadf[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','y1','y2','y3']])

In [6]:
# menghitung pvalue dan menambahkan kolom 'pvalue' pada dataframe
from scipy.stats import chi2
mydatadf ['pvalue'] = 1 - chi2.cdf(mydatadf['mahalanobis'], 14)
#angka terakhir adalah nilai df (k-1, jumlah variabel dikurangi 1)

In [7]:
mydatadf.head(6).apply(lambda s: s.apply('{0:.3f}'.format))
# agar menampilkan 3 angka belakang decimal
# nilai pvalue kurang dari 0,01 menandakan tidak univariate normal

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,y1,y2,y3,mahalanobis,pvalue
0,5.000,5.000,6.000,6.000,5.000,4.000,6.000,7.000,5.000,5.000,6.000,7.000,7.000,7.000,7.000,20.568,0.113
1,6.000,6.000,5.000,6.000,6.000,6.000,6.000,6.000,6.000,4.000,7.000,6.000,7.000,7.000,7.000,19.668,0.141
2,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000,7.000,7.000,7.000,7.000,6.000,6.000,7.468,0.915
3,1.000,2.000,1.000,1.000,3.000,2.000,1.000,1.000,1.000,1.000,1.000,2.000,1.000,1.000,1.000,64.538,0.000
4,7.000,6.000,7.000,7.000,7.000,7.000,6.000,7.000,7.000,7.000,7.000,7.000,7.000,7.000,7.000,4.586,0.991
5,7.000,7.000,7.000,6.000,6.000,7.000,6.000,7.000,6.000,6.000,6.000,6.000,7.000,6.000,7.000,14.647,0.403


In [8]:
# MULTIVARIATE
# mengembalikan df ke semula (tanpa mahalanobis dan pvalue)
mydata = mydatadf.drop(['mahalanobis', 'pvalue'], axis=1)

In [9]:
from pingouin import multivariate_normality
multivariate_normality(mydata, alpha=0.05)

HZResults(hz=2.416839234174424, pval=0.0, normal=False)

# MENGUJI MULTIKOLINEARITAS

In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# membentuk df khusus independen variabel
IV = mydata[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12']]

# membentuk df VIF 
vif_data = pd.DataFrame()
vif_data["Variabel"] = IV.columns

# calculating VIF for each variabel
vif_data["VIF"] = [variance_inflation_factor(IV.values, i)
                          for i in range(len(IV.columns))]
  
print(vif_data)

   Variabel         VIF
0        x1  153.203815
1        x2  100.100612
2        x3  180.344208
3        x4  305.578200
4        x5   99.665656
5        x6   54.414202
6        x7   35.010172
7        x8   62.700885
8        x9   85.616063
9       x10   75.718328
10      x11   65.232376
11      x12   99.275880


# MEMBENTUK MODEL SEM

In [13]:
# mendefinisikan model
mymodel = """
advising =~ x1 + x2 + x3 + x4
tutoring =~ x5 + x6 + x7 + x8
value =~ x9 + x10 + x11 + x12
satisfaction =~ y1 + y2 + y3
value ~ tutoring + advising
satisfaction ~ value + tutoring + advising
"""

In [14]:
# membentuk model
modelsem = sem.Model(mymodel)

In [15]:
# karena non-normality, coba digunakan UWLS sebagai metode estimasi
modelsem.fit(mydata, obj="DWLS", solver="SLSQP")

SolverResult(fun=0.0984083502603221, success=True, n_it=37, x=array([0.94583307, 1.0374168 , 1.02769678, 1.03273332, 1.04389717,
       1.04155908, 1.04406939, 1.04116147, 0.90657204, 1.37369263,
       1.28220697, 0.44391754, 0.52207114, 0.26190709, 0.09810194,
       0.30978922, 0.23182868, 0.29644596, 0.22558752, 0.67670619,
       0.09091025, 0.59317693, 0.61103438, 0.76284123, 0.35481795,
       0.15487113, 0.11703119, 0.28829946, 0.49506011, 0.78834245,
       0.54787885, 0.28546457, 0.10464904, 0.72212477, 0.37979257,
       0.71343754]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='DWLS')

In [16]:
# memprediksi factor loadings
factors = modelsem.predict_factors(mydata)
print(factors.head())

   advising  satisfaction  tutoring     value
0 -0.483999      0.343102 -0.261868  0.306718
1 -0.497077      0.339702  0.235974  0.078655
2 -0.285980     -0.102543  0.236842  0.860217
3 -4.978837     -4.052877 -3.533522 -4.636766
4  0.551673      0.494186  0.985566  1.149691


In [17]:
# mengoptimasi model
opt = sem.Optimizer(modelsem)
obj = opt.optimize()

In [18]:
from semopy.inspector import inspect
inspect(opt)

,lval,op,rval,Estimate,Std. Err,z-value,p-value
0,value,~,tutoring,0.423136,0.122507,3.453971,0.000552
1,value,~,advising,0.542297,0.11915,4.551387,0.000005
2,satisfaction,~,value,0.225493,0.036697,6.14475,0.0
3,satisfaction,~,tutoring,0.095008,0.050175,1.893557,0.058284
4,satisfaction,~,advising,0.334220,0.053333,6.266602,0.0
5,x1,~,advising,1.000000,-,-,-
6,x2,~,advising,0.891975,0.066201,13.473657,0.0
7,x3,~,advising,0.973476,0.057095,17.050164,0.0
8,x4,~,advising,1.012596,0.050903,19.892791,0.0
9,x5,~,tutoring,1.000000,-,-,-


In [19]:
# Model fit
stats = sem.calc_stats(modelsem)
print(stats.T)

                     Value
DoF              84.000000
DoF Baseline    105.000000
chi2             17.811911
chi2 p-value      1.000000
chi2 Baseline  1711.124065
CFI               1.041210
GFI               0.989591
AGFI              0.986988
NFI               0.989591
TLI               1.051512
RMSEA             0.000000
AIC              69.041533
BIC             184.187426
LogLik            1.479233


In [21]:
# membentuk grafik SEM
gg = sem.semplot(modelsem, filename = "semedugg.png")
gg

In [23]:
# Create Report
from semopy import ModelMeans
from semopy import report
modelsem2 = ModelMeans(mymodel)
modelsem2.fit(mydata)
report(modelsem2, "Education Report SEM")

# END